In [1]:
import pandas as pd
import numpy as np
import ast
import datetime

In [2]:
df = pd.read_csv("Boder_languages_per_line_final1.csv")

In [3]:
# strange: things are not in list, but in str...
# have to resplit
df.german = df.german.apply(lambda x: ast.literal_eval(x))
df.french = df.french.apply(lambda x: ast.literal_eval(x))
df.english = df.english.apply(lambda x: ast.literal_eval(x))
df.spanish = df.spanish.apply(lambda x: ast.literal_eval(x))
df.polish = df.polish.apply(lambda x: ast.literal_eval(x))
df.russian = df.russian.apply(lambda x: ast.literal_eval(x))
df.lithuanian = df.lithuanian.apply(lambda x: ast.literal_eval(x))
df.yiddish = df.yiddish.apply(lambda x: ast.literal_eval(x))
df.others = df.others.apply(lambda x: ast.literal_eval(x))

In [4]:
# make all the time points in a single column
df["times"]=df.german+df.french+df.english+df.spanish+df.polish+df.russian+df.lithuanian+df.yiddish+df.others

In [5]:
df.times=df.times.apply(lambda x: sorted(x))

In [6]:
# start should be every time point except for the last
# end should be every time point except for the first
# length should be end-start
# need to change back to get time_delta for now

# start excludes the last point
df["start_points"]=df.times.apply(lambda x: x[:-1])

# end excludes the first point
df["end_points"]=df.times.apply(lambda x: x[1:])

# length is end - start
time_length_list=[]
for i in range(len(df)):
    time_length_list.append([b_i - a_i for a_i, b_i in zip(df.start_points[i], df.end_points[i])])
df["time_length"]=time_length_list

In [7]:
# change to datetime
# try format xx:xx:xx
# helper function
# change a list to datetime
# change seconds into whole seconds
def to_datetime (a):
    b = [] # output
    for i in a:
        b.append(str(datetime.timedelta(seconds=i)))
    return b

In [8]:
df.german = df.german.apply(lambda x: to_datetime(x))
df.french = df.french.apply(lambda x: to_datetime(x))
df.english = df.english.apply(lambda x: to_datetime(x))
df.spanish = df.spanish.apply(lambda x: to_datetime(x))
df.polish = df.polish.apply(lambda x: to_datetime(x))
df.russian = df.russian.apply(lambda x: to_datetime(x))
df.lithuanian = df.lithuanian.apply(lambda x: to_datetime(x))
df.yiddish = df.yiddish.apply(lambda x: to_datetime(x))
df.others = df.others.apply(lambda x: to_datetime(x))
df.times = df.times.apply(lambda x: to_datetime(x))
df.start_points = df.start_points.apply(lambda x: to_datetime(x))
df.end_points = df.end_points.apply(lambda x: to_datetime(x))

In [9]:
### make data tidy
# create new df: df1
# this df1 is tidy for Tableau
df1 = pd.DataFrame(columns=["id","language","start","end","length"])

In [10]:
# lists for df1
# excpet for language
i_start_points=[]
i_end_points=[]
i_time_length=[]
i_id=[]
for i in range(len(df)):
    i_start_points+=df.start_points[i]
    i_id+=[df.id[i]]*len(df.start_points[i])
    i_end_points+=df.end_points[i]
    i_time_length+=df.time_length[i]

In [11]:
# copy lists to df1
df1.id = i_id
df1.start = i_start_points
df1.end = i_end_points
df1.length = i_time_length

In [12]:
# write inside of df is the best choice
# new column called languages
df["languages"]=[[]]*len(df)

In [13]:
list_full = []
for i in range(len(df)):
    for j in df.start_points[i]:
        for k in ["german", "french", "english", "spanish", "polish", "russian", "lithuanian", "yiddish", "others"]:
            if j in df[k][i]:
                list_full.append(k)

In [14]:
df1.language = list_full

In [15]:
df1.to_csv("Boder_languages_per_line_final_preprocessed.csv")